In [ ]:
import einops as ein
from einops.layers.torch import Rearrange

import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision.utils import save_image
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD # -ELBO

In [ ]:
class Diffusion (nn.Module):
    def __init__(self, noise_steps=100, latent_dim=2):
        super (Diffusion, self).__init__()
        self.beta = ein.repeat (torch.tensor ([0.05]), "() -> a", a=noise_steps)
        self.alpha = 1 - self.beta
        self.alpha_bar = torch.cumprod (self.alpha, dim=0)

    def q (self, x0, t):
        temp = self.alpha_bar[t]
        mean = torch.sqrt (temp) * x0
        var = 1 - temp
        return mean, var
        
    def sample_q (self, x0, t, epsilon=None):
        if epsilon is None:
            epsilon = torch.randn_like (x0)

        mean, var = self.q (x0, t)
        return mean + torch.sqrt (var) * epsilon

    def noise_forward (self, data):
        return 0

    def noise_backward (self, noise):
        return 0

    

In [ ]:
model = Diffusion ()
#print (model.beta)
#print (model.noise_forward(1))
#print (model.alpha_bar)
#print (model.q (torch.tensor ([1,2,3]), [0,0,1]))
print (model.sample_q (torch.tensor ([1.,2.,3.]), [0,0,1]))